## Illustrative example implementing COSMOS ideas on Korda2020 optimisation problem

In the paper by Korda2020 we are solving 
$$ \min_{\lambda_i, g_i} \left\|
  \begin{bmatrix}
      h_i(x^1(0)) & h_i(x^2(0)) & \cdots \\ 
      h_i(x^1(1)) & h_i(x^2(1)) & \cdots \\ 
      h_i(x^1(2)) & h_i(x^2(2)) & \cdots \\ 
      \vdots & \vdots & \ddots %\\ h_i(x^1(N)) \\ h_i(x^2(0)) \\ \vdots
  \end{bmatrix}- 
  \begin{bmatrix}
      1 & 1 & 1 & \cdots \\
      \lambda_{1} & \lambda_{2} & \lambda_{3} & \cdots \\
      \lambda_{1}^2 & \lambda_{2}^2 & \lambda_{3}^2 & \cdots \\
      \vdots & \vdots & \vdots & \ddots
  \end{bmatrix}
  \begin{bmatrix}% End of phantom section for vertical brace alignment
      g_{1} (x^1(0)) & g_{1} (x^2(0)) & \cdots \\ 
      g_{2} (x^1(0)) & g_{2} (x^2(0)) & \cdots \\
      \vdots & \vdots & \ddots
  \end{bmatrix} \right\| $$

That is, given a matrix $Y$ we seek a decomposition into a product of a Vandermonde matrix $\Lambda$ and a matrix of initial states $G$, formulated as a least-squares problem $\| Y - \Lambda G \|$. The idea is to turn this problem from a nonconvex problem into a convex problem with rank constraint, and apply the ideas in COSMOS to this work. 
Defining $D_\lambda = \text{diag}(\lambda_1, \lambda_2, \dots)$, and $\overline \Lambda$ as the matrix $\Lambda$, but shifted one place up, we obtain that $\overline \Lambda = \Lambda D_{\lambda}$. These relationships can be written in terms of a rank constraint, for which we introduce
$$ H = \begin{bmatrix} \hat M & \Lambda & \overline \Lambda \\ \hat G & I_{N_g} & D_\lambda \end{bmatrix}. $$

By the Schur decomposition, the rank of the matrix $\hat M - \Lambda I_{N_g} \hat G$ is equivalent to rank of the left $2 \times 2$ block submatrix of $H$. Similarly, the introduced relationship between $\Lambda$ and its shifted matrix $\overline \Lambda$, gives the latter columns the same rank. Hence we obtain the optimisation problem
$$ \min \| Y - \hat M \| $$
subject to $\text{rank}(H) = N_g$


The code below is an iterative method, solving 
$$ \text{argmin}_{\hat M, \hat G, \Lambda, \overline \Lambda, \lambda_i} \| Y - \hat M \|_2 + \rho \left( \| H \|_* - \text{tr} \left( U_1^T H V_1 \right) \right) $$
subject to: 
\\[\Lambda[0,:] = [1, 1, \dots] \\
  \overline \Lambda[0:end-1, :] = \Lambda[1:,:] \\
  \Lambda[1,:] = [\lambda_1, \lambda_2, \dots]\\]
iteratively, using the truncated singular value decomposition for $H = U S V^*$. 

In [6]:
import cvxpy as cvx
import numpy as np
np.random.seed(123)

def H_Theta(M, Theta, ThetaShift, X, Id, L):
    '''Function for constructing the blockmatrix in the rank constraint optimisation problem
    Using the input matrices, the following block matrix is constructed:
    | M  Theta  ThetaShift |
    | X    Id     L        |
    '''
    return cvx.bmat([[M, Theta, ThetaShift], [X, Id, L]])


N = 5                             # Number of rows of measurement data Y, i.e. measurement sequence length
N_t = 20                           # Number of columns of measurement data Y, i.e. number of measurement sequences
N_g = 3                            # Rank used as rank constraint, and to construct measurement data Y
Id = np.eye(N_g)                   # Usefull identity matrix
# Total number of optimisation variables = N_t * N_g + N_g

# Define Y as decomposed product of a VanderMonde matrix and random initial states.
Y = np.vander([0.9, 0.95, 0.8], N, increasing=True).T @ np.random.rand(N_g, N_t) 

# Initialise U and V as empty matrices
U_j = [np.zeros((N + N_g, N_g))]
V_j = [np.zeros((N_t + 2 * N_g, N_g))]

# Initialise CVX optimisation problem
Mhat = cvx.Variable(Y.shape)
Ghat = cvx.Variable((N_g, N_t))
ThetaHat = cvx.Variable((N,N_g))
ThetaHatShift = cvx.Variable((N, N_g))
Lhat = cvx.diag(cvx.Variable(N_g, complex=False))      

# Constraints: 
#        Requirement that first row of Theta[0,:] = [ 1, 1, 1, ...]
#        Requirement that second row of Theta[1,:] = ThetaShift[0,:]
#        Requirement that second row of Theta[1,:] = [lambda_1, lambda_2, ...]
cons = [ThetaHat[0,:] == np.ones((N_g,)), ThetaHat[1:,:] == ThetaHatShift[:-1,:], ThetaHat[1,:] == cvx.diag(Lhat)]

# Construct rank constraint matrix
Fullmatrix = H_Theta(Mhat, ThetaHat, ThetaHatShift, Ghat, Id, Lhat)

# Initialise variables that change on optimisation iteration
U = cvx.Parameter((N + N_g, N_g), complex=False)
V = cvx.Parameter((N_t + 2 * N_g, N_g), complex=False)
rho = cvx.Parameter(pos=True)

# Define objective function and problem handle
obj = cvx.norm(Y - Mhat,2) +  rho * ( cvx.norm(Fullmatrix, "nuc") - (cvx.trace(U.T @ Fullmatrix @ V))) 
prob = cvx.Problem(cvx.Minimize(obj), cons)

# Initialise optimisation iteration
i = 0
eps = 1
rho_new = 0.01       # Initial value for rho
mu = 1.02            # Growth factor for rho
rho_max = 10     # Maximum value for rho
prev_value = 1e5

# Iterate while error is large and number of iterations is small enough
while eps > 1e-4 and i < 10:
    rho_new = min(mu * rho_new, rho_max)
    
    # Set CVX problem parameters
    rho.value = rho_new
    U.value = U_j[i]
    V.value = V_j[i]
    
    # Solve optimisation problem iteration using cvx
    prob.solve(verbose=False)

    # Compute SVD of rank-constraint matrix value
    Res = Fullmatrix.value # H_Theta_num(M, Theta, ThetaShift, X, Id, L)
    U1, s, V1 = np.linalg.svd(Res, full_matrices=True)

    U_j.append(U1[:,:N_g])
    V_j.append(V1[:N_g,:].T)
 
    # Update error value, and print update
    eps = abs((obj.value - prev_value) / obj.value)
    prev_value = obj.value
    i = i + 1
    print(f'Iteration {i} completed with error {obj.value:.3f} and rho={rho_new:.2f}')
    
# Extract solutions from optimisation procedure
M = Mhat.value
G = Ghat.value
Theta = ThetaHat.value
ThetaShift = ThetaHatShift.value
L = Lhat.value 

print()
print(f'------------------ Finished optimisation procedure with status word: {prob.status} ----------------------------')
print(f'Found eigenvalue sequence                                  {np.array2string(np.diag(L), precision=2)}')
# print(Theta)
# print(ThetaShift)
print(f"Final nuclear norm of rank-constraint matrix               {np.linalg.norm(Res, 'nuc'):.3f}")
print(f'Least-square norm error ||Y - M||                          {np.linalg.norm(Y - M, 2):.3f}')
print(f'Least-square norm error ||Y - L*G||                        {np.linalg.norm(Y - np.vander(np.diag(L), N, increasing=True).T @ G, 2):.3f}') 
# print(Res)

Iteration 1 completed with error 0.170 and rho=0.01
Iteration 2 completed with error 0.012 and rho=0.01
Iteration 3 completed with error 0.012 and rho=0.01
Iteration 4 completed with error 0.012 and rho=0.01
Iteration 5 completed with error 0.011 and rho=0.01
Iteration 6 completed with error 0.012 and rho=0.01
Iteration 7 completed with error 0.012 and rho=0.01
Iteration 8 completed with error 0.012 and rho=0.01
Iteration 9 completed with error 0.012 and rho=0.01
Iteration 10 completed with error 0.012 and rho=0.01

------------------ Finished optimisation procedure with status word: optimal ----------------------------
Found eigenvalue sequence                                  [0.74 0.01 0.04]
Final nuclear norm of rank-constraint matrix               16.870
Least-square norm error ||Y - M||                          0.000
Least-square norm error ||Y - L*G||                        10.005


In [7]:
print(np.vander(np.diag(L), N, increasing=True).T @ G)
print(Y - np.vander(np.diag(L), N, increasing=True).T @ G)
# print(Y - M)

[[ 0.66403187  0.13799901  0.28469454  0.45899238  0.85754551  0.30486178
   0.64588184  0.96166547  0.53413857  0.56828548  0.18937231  0.84072217
   0.58207843  0.42535756  0.36603066  0.4377042   0.36263773  0.54922027
   0.80179837  0.53304959]
 [ 0.33882806 -0.04310725  0.08587414  0.20419406  0.46507379  0.14318923
   0.34316924  0.6141524   0.30958662  0.29280128  0.09840673  0.4934948
   0.32797629  0.2338007   0.17479192  0.21739443  0.18857643  0.25776005
   0.40703408  0.27512184]
 [ 0.24575135 -0.03694164  0.05932875  0.14659792  0.33870634  0.10322796
   0.24954926  0.45163256  0.22639764  0.21252895  0.07144897  0.36116321
   0.23944169  0.17044832  0.12617741  0.15735222  0.13696963  0.18586996
   0.29514658  0.19970403]
 [ 0.1816857  -0.02751545  0.04375591  0.10832674  0.25045806  0.07629453
   0.18451666  0.33411793  0.16744413  0.15713007  0.05282531  0.26712685
   0.17707718  0.12604531  0.0932623   0.11631977  0.10126978  0.13737629
   0.21820147  0.14764831]
 [ 0.